In [1]:
%install-location $cwd/swift-install
%install '.package(path: "$cwd/SchwiftyNotebook_batch_boi")' SchwiftyNotebook_batch_boi

Installing packages:
	.package(path: "/home/ubuntu/workspace/fast-ai-swift/SchwiftyNotebook_batch_boi")
		SchwiftyNotebook_batch_boi
With SwiftPM flags: []
Working in: /tmp/tmpqxsoc0k8/swift-install
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/envs/swift-env/lib/libuuid.so.1: no version information availabl

In [2]:
//export
import Path
import TensorFlow
import SchwiftyNotebook_batch_boi

In [3]:
//export
public typealias SingleValueAndGradientChain<T>=(value: T, gradientChain: (T) -> T )
public typealias SingleInputDifferentiable<T> = (T) -> SingleValueAndGradientChain<T>

In [4]:
//export
public let identititySVGC: SingleInputDifferentiable<TensorFloat> = { x in (value: x, gradientChain: { y in y }) }

In [5]:
//export
public struct HyperParameter {
    let learningRate: Float
    
    init(learningRate: Float){
        self.learningRate = learningRate
    }
}

In [6]:
//export
public struct RuntimeError: Error {
    let message: String

    init(_ message: String) {
        self.message = message
    }

    public var localizedDescription: String {
        return message
    }
}

In [7]:
//export
public typealias ThreeInputGradient = (TensorFloat) -> (TensorFloat, TensorFloat, TensorFloat)

public struct DenseLayer {
    let activationFunction: SingleInputDifferentiable<TensorFloat>
    let weightParameter: TensorFloat
    let biasParameter: TensorFloat
    private let gradientChain: ThreeInputGradient

    private init(activationFunction: @escaping SingleInputDifferentiable<TensorFloat>,
                 weightParameter: TensorFloat,
                 biasParameter: TensorFloat,
                 gradientChain: @escaping ThreeInputGradient) {
        self.activationFunction = activationFunction
        self.weightParameter = weightParameter
        self.biasParameter = biasParameter
        self.gradientChain = gradientChain
    }

    private init(denseLayer: DenseLayer,
                 gradientChain: @escaping ThreeInputGradient) {
        self.activationFunction = denseLayer.activationFunction
        self.weightParameter = denseLayer.weightParameter
        self.biasParameter = denseLayer.biasParameter
        self.gradientChain = gradientChain
    }

    init(inputSize: Int,
         outputSize: Int,
         activationFunction: @escaping SingleInputDifferentiable<TensorFloat> = identititySVGC){
        self.activationFunction = activationFunction
        self.weightParameter = TensorFloat(kaimingUniform: TensorShape([inputSize, outputSize]))
        self.biasParameter = TensorFloat(zeros: [outputSize])
        self.gradientChain = { (x: TensorFloat)  in (TensorFloat([1]), TensorFloat([1]), TensorFloat([1]))}
    }

    func forwardPass(inputTensor: TensorFloat) -> (TensorFloat, DenseLayer) {
        let (parameterOutput, parameterGradientChain) =
          linearCombinationAndGradient(inputTensor: inputTensor,
                                       weightParameter: self.weightParameter,
                                       biasParameter: self.biasParameter)
        let (activations, activationsGradientChain) = activationFunction(parameterOutput)

        return (activations, DenseLayer(denseLayer: self,
                                       gradientChain: {ddx in parameterGradientChain(activationsGradientChain(ddx))}))
    }

    public func callAsFunction(_ inputTensor: TensorFloat) -> TensorFloat {
        let parameterOutput =
          linearCombination(inputs: inputTensor,
                                       weights: self.weightParameter,
                                       bias: self.biasParameter)
        let (activations, activationsGradientChain) = activationFunction(parameterOutput)
        return activations
    }

    func backwardPass(ddx: TensorFloat, hyperParams: HyperParameter) -> (TensorFloat, DenseLayer) {
        let (ddxInput, ddxParameter, ddxBias) = self.gradientChain(ddx)
        return (ddxInput, DenseLayer(activationFunction: activationFunction,
                                    weightParameter: self.weightParameter - hyperParams.learningRate * ddxParameter,
                                    biasParameter: self.biasParameter - hyperParams.learningRate * ddxBias,
                                    gradientChain: self.gradientChain))
    }
}

In [30]:
let usedCarDataBunch = fetchUsedCarDataBunch(validationSize: 0.001, batchSize:1024)

In [31]:
let firstBatch = usedCarDataBunch.trainingDataset.first!
let numberFeatures = firstBatch.features.shape[1]
print(firstBatch.features.max(alongAxes: 0))
numberFeatures

[[0.82608694,        1.0,        1.0,        1.0,        1.0,        1.0,  0.9990104, 0.09999999]]


8


In [32]:
let carPriceLayer = DenseLayer(inputSize: numberFeatures, 
                               outputSize: 1, 
                               activationFunction: reLUAndGradient)

In [17]:
let preds = carPriceLayer(firstBatch.features)

In [18]:
(preds.shape, firstBatch.labels.shape)

▿ 2 elements
  ▿ .0 : [1024, 1]
    ▿ dimensions : 2 elements
      - 0 : 1024
      - 1 : 1
  ▿ .1 : [1024]
    ▿ dimensions : 1 element
      - 0 : 1024


In [19]:
let (lossyBoi, gradientChain) = meanSquaredErrorAndGradient(yHat: preds, 
                                                            y: firstBatch.labels.expandingShape(at: 1))

In [20]:
lossyBoi

8.737265e+12


In [21]:
var currentDenseLayer = carPriceLayer
let trialsHyperParameter = HyperParameter(learningRate: 1e-2)
timeTrials(trials: 2) { 
    let (alexYHat, alexLayer) = currentDenseLayer.forwardPass(inputTensor: firstBatch.features)
    let (loss, lossGradientSupplier) = meanSquaredErrorAndGradient(yHat: alexYHat, 
                                                                   y:firstBatch.labels.expandingShape(at: 1))
    let (ddxAlexLayer, enhancedLayer) = alexLayer.backwardPass(ddx: lossGradientSupplier(), 
                                                               hyperParams: trialsHyperParameter)
    currentDenseLayer = enhancedLayer
}

average: 2.734093 ms,   min: 2.695993 ms,   max: 2.772193 ms


In [22]:
let (nextLoss, _) = meanSquaredErrorAndGradient(yHat: currentDenseLayer(firstBatch.features), 
                                                            y: firstBatch.labels.expandingShape(at: 1))
nextLoss

8.449959e+12


In [23]:
var currentDenseLayer = carPriceLayer
let trialsHyperParameter = HyperParameter(learningRate: 1e-2)
let firstValidation = usedCarDataBunch.validationDataset.first!
let (trainingLoss, _) = meanSquaredErrorAndGradient(yHat: currentDenseLayer(firstValidation.features), 
                                                            y: firstValidation.labels.expandingShape(at: 1))
print("First Loss: \(trainingLoss)")
withTime {
    timeTrials(trials: 200) { 
        for batch in usedCarDataBunch.trainingDataset {
            let (alexYHat, alexLayer) = currentDenseLayer.forwardPass(inputTensor: batch.features)
            let (loss, lossGradientSupplier) = meanSquaredErrorAndGradient(yHat: alexYHat, 
                                                                           y:batch.labels.expandingShape(at: 1))
            let (ddxAlexLayer, enhancedLayer) = alexLayer.backwardPass(ddx: lossGradientSupplier(), 
                                                                       hyperParams: trialsHyperParameter)
            currentDenseLayer = enhancedLayer   
        }
    }
}
let (postLoss, _) = meanSquaredErrorAndGradient(yHat: currentDenseLayer(firstValidation.features), 
                                                            y: firstValidation.labels.expandingShape(at: 1))
print("Post Loss: \(postLoss)")
assert(trainingLoss > postLoss)

First Loss: 1.2181415e+12
average: 90.707683165 ms,   min: 83.770458 ms,   max: 95.742902 ms
elapsed time: 18.233004417 s
Post Loss: 4.4469322e+11


In [25]:
let predictedPrices = currentDenseLayer(firstValidation.features)
let error = firstValidation.labels.expandingShape(at: 1) - predictedPrices

In [26]:
(error.mean(), error.standardDeviation())

▿ 2 elements
  - .0 : 350.84937
  - .1 : 666853.1


In [27]:
(predictedPrices[6],  firstValidation.labels[6])

▿ 2 elements
  - .0 : [1148750.5]
  - .1 : 450000.0


In [33]:
var currentDenseLayer = carPriceLayer
let trialsHyperParameter = HyperParameter(learningRate: 1e-1)
withTime {
    var trial = 0
    timeTrials(trials: 200) { 
        var accumulatedTrainingLoss = TensorFloat([0])
        var trainingItems = TensorFloat([0])
        for batch in usedCarDataBunch.trainingDataset {
            let (alexYHat, alexLayer) = currentDenseLayer.forwardPass(inputTensor: batch.features)
            
            let (loss, lossGradientSupplier) = meanSquaredErrorAndGradient(yHat: alexYHat, 
                                                                           y:batch.labels.expandingShape(at: 1))
            accumulatedTrainingLoss += loss
            trainingItems += 1
            
            let (ddxAlexLayer, enhancedLayer) = alexLayer.backwardPass(ddx: lossGradientSupplier(), 
                                                                       hyperParams: trialsHyperParameter)
            currentDenseLayer = enhancedLayer   
        }
        
        
        var accumulatedValidationLoss = TensorFloat([0])
        var validationItems = TensorFloat([0])
        for validationBatch in usedCarDataBunch.validationDataset {
            let (validationLoss, _) = meanSquaredErrorAndGradient(yHat: currentDenseLayer(validationBatch.features), 
                                                                  y: validationBatch.labels.expandingShape(at: 1))
            
            accumulatedValidationLoss += validationLoss
            validationItems += 1
        }
        
        if(trial % 10 == 0) {
            print("Training Loss: \(accumulatedTrainingLoss/trainingItems)")
            print("Validation Loss: \(accumulatedValidationLoss/validationItems)")   
        }
        trial += 1
    }
}

Training Loss: [2.3739564e+12]
Validation Loss: [4.641585e+11]
Training Loss: [2.2129082e+12]
Validation Loss: [3.4123973e+11]
Training Loss: [2.198596e+12]
Validation Loss: [3.201808e+11]
Training Loss: [2.1932645e+12]
Validation Loss: [3.1377152e+11]
Training Loss: [2.19018e+12]
Validation Loss: [3.0992076e+11]
Training Loss: [2.1879532e+12]
Validation Loss: [3.0667804e+11]
Training Loss: [2.1862121e+12]
Validation Loss: [3.0377053e+11]
Training Loss: [2.18491e+12]
Validation Loss: [3.0116672e+11]
Training Loss: [2.1839029e+12]
Validation Loss: [2.9879216e+11]
Training Loss: [2.1830917e+12]
Validation Loss: [2.9652828e+11]
Training Loss: [2.1823935e+12]
Validation Loss: [2.9404388e+11]
Training Loss: [2.1814592e+12]
Validation Loss: [2.9130097e+11]
Training Loss: [2.1805977e+12]
Validation Loss: [2.8902153e+11]
Training Loss: [2.1798666e+12]
Validation Loss: [2.8716234e+11]
Training Loss: [2.1792543e+12]
Validation Loss: [2.8550863e+11]
Training Loss: [2.1787401e+12]
Validation Loss:

In [29]:
exportSchwiftyNotebook(notebookName: "Layer_Refined")

success
